In [1]:
import pandas as pd
import numpy as np
import json

# Data Cleaning and Rough Draft for ACFT Calculaor 

## I took excel tables from here: https://companyleader.themilitaryleader.com/2019/11/07/acft-calculator-tracker/
## That are accurate to the US Army PDF that's more difficult to take data from here: https://www.goarmy.com/how-to-join/requirements/fitness

### Being new to Swift, I chose to write a rough draft version of the main calculations in Python. Knowing that Swift seems to favor funciotnal programming I went with dictionaries over Python objects. This also made sense with the end goal being to place the data in JSON (since JavaScript objects are pretty similar to Python dictionaries)

### I felt the author, "The Company Leader", of the Excel calculator was too faithful to the Army website's formatting. I'm taking his excel tables, turning them to pandas data frames, subsetting them as needed to make the JSON file for the actual app. There's also a function to do the primary logic of the iOS app that takes a dictionary as an argument and calculate an indivual's score. 

### There are three stages to this app: (1) basic iOS ACFT calculator for an individual, (2) add SQLite for tracking individual historic performance metrics and for unit leaders to quickly log results for their soldiers and recieve an Excel document with the results, and (3) create a feature to scan handwriting on DA 705s to instantly get ACFT scores.  This notebook is only relevant for stage one of this app. 

## converting excel tables to data frames to be cleaned for each event 

 ### copied and pasted score charts into my own excel file and creating data frames from there 

In [2]:
# the big adjustments are really just setting this up to be one bit list of dictionaries for a JSON document 
# I added gender, so that it'll be easy to filter later 

# male scoring charts
male_dl = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_dl").replace('---', np.nan).assign(gender='male').assign(event='deadlift')
male_spt = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_spt").replace('---', np.nan).assign(gender='male').assign(event='standing power throw')
male_hrp = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_hrp").replace('---', np.nan).assign(gender='male').assign(event='hand release pushups')
male_sdc = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_sdc").replace('---', np.nan).assign(gender='male').assign(event='sprint drag carry')
male_plk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_plk").replace('---', np.nan).assign(gender='male').assign(event='plank')
male_tmr = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_tmr").replace('---', np.nan).assign(gender='male').assign(event='two mile run')

# female scoring charts
female_dl = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_dl").replace('---', np.nan).assign(gender='female').assign(event='deadlift')
female_spt = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_spt").replace('---', np.nan).assign(gender='female').assign(event='standing power throw')
female_hrp = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_hrp").replace('---', np.nan).assign(gender='female').assign(event='hand release pushups')
female_sdc = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_sdc").replace('---', np.nan).assign(gender='female').assign(event='sprint drag carry')
female_plk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_plk").replace('---', np.nan).assign(gender='female').assign(event='plank')
female_tmr = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_tmr").replace('---', np.nan).assign(gender='female').assign(event='two mile run')

# male alternative cardio
male_walk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_walk").assign(gender='male').assign(event='walk')
male_swim = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_swim").assign(gender='male').assign(event='swim')
male_bike = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_bike").assign(gender='male').assign(event='bike')
male_row = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_row").assign(gender='male').assign(event='row')

# female alternative cardio
female_walk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_walk").assign(gender='female').assign(event='walk')
female_swim = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_swim").assign(gender='female').assign(event='swim')
female_bike = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_bike").assign(gender='female').assign(event='bike')
female_row = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_row").assign(gender='female').assign(event='row')


sheets = [ # list of all data frames 
    # core six events for both genders
    male_dl, female_dl, male_spt, female_spt, 
    male_hrp, female_hrp, male_sdc, female_sdc, 
    male_plk, female_plk, male_tmr, female_tmr,
    # alternative cardio events... worry about them later 
    male_walk, male_swim, male_bike, male_row,
    female_walk, female_swim, female_bike, female_row
]

In [3]:
female_sdc

,17,22,27,32,37,42,47,52,57,62,Points,gender,event
0,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,0,female,sprint drag carry
1,415,415,415,422,427,442,451,503,548,548,0,female,sprint drag carry
2,414,414,414,421,426,441,450,502,547,547,1,female,sprint drag carry
3,413,413,413,420,425,440,449,501,546,546,2,female,sprint drag carry
4,412,412,412,419,424,439,448,500,545,545,3,female,sprint drag carry
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,206,205,206,210,214,218,226,232,241,241,96,female,sprint drag carry
98,205,202,204,208,211,217,224,230,239,239,97,female,sprint drag carry
99,202,200,201,205,210,215,222,228,234,234,98,female,sprint drag carry
100,159,156,157,201,204,210,213,221,228,228,99,female,sprint drag carry


### filtering data frames and creating a JSON file 

#### exploratory data analysis 

In [4]:
# probably worth keeping a custom points vector with each data frame, 
# as the number of rows vary 
for df in sheets: # showing that there's an even amount of entries for both genders
    print(len(df))

48
48
102
102
48
48
102
102
102
102
102
102
2
2
2
2
2
2
2
2


### in this cell I've placed all raw scores (times, lbs/reps lifted, etc), points, and events for all genders and ages as dictionaries into one single flat list. This isn't the format that I want to keep for the JSON object, but it's easy to work with later 

In [6]:
# ages are the same for all data frames in sheets
ages = female_dl.columns.tolist()[:-3] # ['17', '22', '27', '32', '37', '42', '47', '52', '57', '62']  

age_raw_pts_dicts = []  # List to store the dictionaries

for sheet in sheets:
    points = sheet['Points'].tolist()
    gender = sheet['gender'].iloc[0]  # constant for the sheet
    event = sheet['event'].iloc[0]  #  constant for the sheet
    
    for age in ages:
        age_scale = sheet[age].tolist()
        
        for i in range(len(age_scale)):
            if pd.notna(age_scale[i]):  # handles NaN directly
                # appends the dictionary with aligned data
                age_raw_pts_dicts.append({
                    'age': age,
                    'raw': age_scale[i],
                    'points': points[i],
                    'gender': gender,
                    'event': event
                })

In [80]:
age_raw_pts_dicts[:5] # showing the structure...continues for both genders until alternate cardio for the 62>= year old category 

[{'age': '17', 'raw': 0.0, 'points': 0, 'gender': 'male', 'event': 'deadlift'},
 {'age': '17',
  'raw': 80.0,
  'points': 0,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 90.0,
  'points': 10,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 100.0,
  'points': 20,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 110.0,
  'points': 30,
  'gender': 'male',
  'event': 'deadlift'}]

In [81]:
len(age_raw_pts_dicts) # total number of dictionaries in the list 

7847

## this is the first layer of subsetting, not the most efficient way to do this, but it's easy for me to manage and control. the result is having list of dictionaries for scoring for each gender by age group

In [7]:
grouped_data = {}
genders = ['male', 'female']


# making subsets of age_raw_pts_dicts for both readability and to minimize future looping in application
for age in ages:
    for gender in genders:
        key = f"{age}_plus_{gender}"  # dynamic variable name
        grouped_data[key] = [
            d for d in age_raw_pts_dicts 
            if d['age'] == age and d['gender'] == gender
        ]


# males
seventeen_plus_male = grouped_data['17_plus_male']
twentytwo_plus_male = grouped_data['22_plus_male']
twentyseven_plus_male = grouped_data['27_plus_male']
thirtytwo_plus_male = grouped_data['32_plus_male']
thirtyseven_plus_male = grouped_data['37_plus_male']
fortytwo_plus_male = grouped_data['42_plus_male']
fortyseven_plus_male = grouped_data['47_plus_male']
fiftytwo_plus_male = grouped_data['52_plus_male']
fiftyseven_plus_male = grouped_data['57_plus_male']
sixty_two_plus_male = grouped_data['62_plus_male']

males = [seventeen_plus_male, twentytwo_plus_male, twentyseven_plus_male, thirtytwo_plus_male,
         thirtyseven_plus_male, fortytwo_plus_male, fortyseven_plus_male, fiftytwo_plus_male, 
         fiftyseven_plus_male, sixty_two_plus_male]

# females
seventeen_plus_female = grouped_data['17_plus_female']
twentytwo_plus_female = grouped_data['22_plus_female']
twentyseven_plus_female = grouped_data['27_plus_female']
thirtytwo_plus_female = grouped_data['32_plus_female']
thirtyseven_plus_female = grouped_data['37_plus_female']
fortytwo_plus_female = grouped_data['42_plus_female']
fortyseven_plus_female = grouped_data['47_plus_female']
fiftytwo_plus_female = grouped_data['52_plus_female']
fiftyseven_plus_female = grouped_data['57_plus_female']
sixty_two_plus_female = grouped_data['62_plus_female']

females = [seventeen_plus_female, twentytwo_plus_female, twentyseven_plus_female, thirtytwo_plus_female,
         thirtyseven_plus_female, fortytwo_plus_female, fortyseven_plus_female, fiftytwo_plus_female, 
         fiftyseven_plus_female, sixty_two_plus_female]





In [12]:
thirtytwo_plus_female[15:20] # example output. Notice that the event data is still mixed and inefficient for a JSON object 

[{'age': '32',
  'raw': 200.0,
  'points': 96,
  'gender': 'female',
  'event': 'deadlift'},
 {'age': '32',
  'raw': 210.0,
  'points': 98,
  'gender': 'female',
  'event': 'deadlift'},
 {'age': '32',
  'raw': 220.0,
  'points': 99,
  'gender': 'female',
  'event': 'deadlift'},
 {'age': '32',
  'raw': 230.0,
  'points': 100,
  'gender': 'female',
  'event': 'deadlift'},
 {'age': '32',
  'raw': 0.0,
  'points': 0,
  'gender': 'female',
  'event': 'standing power throw'}]

### I wrote a function that separates dictionaries into their own lists to use later 

In [68]:
# a function that will automatically create dictionaries with lists
# this works perfectly if we have it subsetted where the current JSON object is (able to filter gender then age ex: dict_to_json['female']['17'])
# fake event table 
fake_events = [ # not a real events, but points are scaled in similar way 
    # jumping jacks
    {'raw': 0, 'Points': 0, 'event': 'jumping jacks'}, {'raw': 5, 'Points': 0, 'event': 'jumping jacks'}, {'raw': 7, 'Points': 30, 'event': 'jumping jacks'}, 
    {'raw': 10, 'Points': 60, 'event': 'jumping jacks'}, {'raw': 12, 'Points': 70, 'event': 'jumping jacks'}, {'raw': 12, 'Points': 70, 'event': 'jumping jacks'}, 
    {'raw': 20, 'Points': 90, 'event': 'jumping jacks'}, {'raw': 22, 'Points': 95, 'event': 'jumping jacks'}, {'raw': 24, 'Points': 100, 'event': 'jumping jacks'},    
    # low crawl 
    {'raw': 1000, 'Points': 0, 'event': 'low crawl'}, {'raw': 350, 'Points': 0, 'event': 'low crawl'}, {'raw': 240, 'Points': 30, 'event': 'low crawl'}, 
    {'raw': 210, 'Points': 60, 'event': 'low crawl'}, {'raw': 150, 'Points': 70, 'event': 'low crawl'}, {'raw': 120, 'Points': 70, 'event': 'low crawl'}, 
    {'raw': 110, 'Points': 90, 'event': 'low crawl'}, {'raw': 105, 'Points': 95, 'event': 'low crawl'}, {'raw': 100, 'Points': 100, 'event': 'low crawl'},   
]

def make_event_lists(scores): 
    
    # temporary empty lists of fake events that need to be replaced with the real events of: 
    # deadlift, standing power throw, hand release pushups, sprint drag carry, plank, two mile run, row, swim, bike, and walk
    jumping_jacks = [] 
    low_crawl = []
    for score in scores: 
        if score['event'] == 'jumping jacks': 
            jumping_jacks.append(score)
        elif score['event'] == 'low crawl': 
            low_crawl.append(score)

    return {'jumping jacks': jumping_jacks, 'low_crawl': low_crawl}

new_layer = make_event_lists(fake_events)
new_layer['jumping jacks']

[{'raw': 0, 'Points': 0, 'event': 'jumping jacks'},
 {'raw': 5, 'Points': 0, 'event': 'jumping jacks'},
 {'raw': 7, 'Points': 30, 'event': 'jumping jacks'},
 {'raw': 10, 'Points': 60, 'event': 'jumping jacks'},
 {'raw': 12, 'Points': 70, 'event': 'jumping jacks'},
 {'raw': 12, 'Points': 70, 'event': 'jumping jacks'},
 {'raw': 20, 'Points': 90, 'event': 'jumping jacks'},
 {'raw': 22, 'Points': 95, 'event': 'jumping jacks'},
 {'raw': 24, 'Points': 100, 'event': 'jumping jacks'}]

## dict_to_json only uses the males and females lists and a dictionary comprehension to separate by age. Again, the smallest subset here is all events for a specific age and gender. Still need to further subset for more efficient searching in a future application 

In [16]:
dict_to_json = {
    'male': {
        ages[i]: males[i] for i in range(len(ages))
    },
    'female': {
        ages[i]: females[i] for i in range(len(ages))
    }
}



In [17]:
dict_to_json['male']['32'][:5] # output of the first 5 events (deadlift failing scores) for men age 32+

[{'age': '32', 'raw': 0.0, 'points': 0, 'gender': 'male', 'event': 'deadlift'},
 {'age': '32',
  'raw': 80.0,
  'points': 0,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '32',
  'raw': 90.0,
  'points': 10,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '32',
  'raw': 100.0,
  'points': 20,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '32',
  'raw': 110.0,
  'points': 30,
  'gender': 'male',
  'event': 'deadlift'}]

In [19]:
dict_to_json['male']['32'][50:55] # same dictionary, but standing power throw minimum passing scores 

[{'age': '32',
  'raw': 6.5,
  'points': 60,
  'gender': 'male',
  'event': 'standing power throw'},
 {'age': '32',
  'raw': 7.1,
  'points': 61,
  'gender': 'male',
  'event': 'standing power throw'},
 {'age': '32',
  'raw': 7.4,
  'points': 62,
  'gender': 'male',
  'event': 'standing power throw'},
 {'age': '32',
  'raw': 7.6,
  'points': 63,
  'gender': 'male',
  'event': 'standing power throw'},
 {'age': '32',
  'raw': 7.9,
  'points': 64,
  'gender': 'male',
  'event': 'standing power throw'}]

## using the previously made make_event_list, I've updated it to mutate dict_to_json to give specificly subsetted results by gender, age and event. It's now easy to handle and obtain the points by the individual's raw score without needing to make future functions/excessive looping

In [20]:


def make_event_lists_updated(scores): 
    # replaced  temporary placeholders with real event names
    event_dict = {event: [] for event in {
        'deadlift', 'standing power throw', 'hand release pushups',
        'sprint drag carry', 'plank', 'two mile run',
        'row', 'swim', 'bike', 'walk'
    }}
    
    for score in scores:
        if score['event'] in event_dict:
            event_dict[score['event']].append(score)
    
    return event_dict

# adding the new event-organized layer
for gender, ages in dict_to_json.items():
    for age, scores in ages.items():
        # organizing scores by event for each age group
        event_layer = make_event_lists_updated(scores)
        dict_to_json[gender][age] = event_layer



In [22]:
dict_to_json['male']['62']['deadlift'] # shows all deadlift score results for a 62+ year old 

[{'age': '62', 'raw': 0.0, 'points': 0, 'gender': 'male', 'event': 'deadlift'},
 {'age': '62',
  'raw': 80.0,
  'points': 0,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62',
  'raw': 90.0,
  'points': 10,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62',
  'raw': 100.0,
  'points': 20,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62',
  'raw': 110.0,
  'points': 30,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62',
  'raw': 120.0,
  'points': 40,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62',
  'raw': 130.0,
  'points': 50,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62',
  'raw': 140.0,
  'points': 60,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62',
  'raw': 150.0,
  'points': 72,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62',
  'raw': 160.0,
  'points': 82,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62',
  'raw': 170.0,
  'points': 92,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '62'

In [76]:
m_per_min = 7000 / 30
m_per_min * 25

5833.333333333334